In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import os

try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
except:
    pass


In [2]:

base_dir = 'E:/Dataset'
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Test')

img_size = (128, 128)
batch_size = 64


In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=15
)

val_test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 30405 images belonging to 2 classes.
Found 21428 images belonging to 2 classes.
Found 6001 images belonging to 2 classes.


In [5]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid', dtype='float32')
])


c:\Users\abc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [7]:

train_data = tf.data.Dataset.from_generator(lambda: train_data, output_signature=(
    tf.TensorSpec(shape=(None, img_size[0], img_size[1], 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)).prefetch(tf.data.AUTOTUNE)

val_data = tf.data.Dataset.from_generator(lambda: val_data, output_signature=(
    tf.TensorSpec(shape=(None, img_size[0], img_size[1], 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)).prefetch(tf.data.AUTOTUNE)


In [ ]:

history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


Epoch 1/10


In [ ]:

test_data = tf.data.Dataset.from_generator(lambda: test_data, output_signature=(
    tf.TensorSpec(shape=(None, img_size[0], img_size[1], 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)).prefetch(tf.data.AUTOTUNE)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")
